# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

$tf_{x,j}=\frac{n_{x}}{N_{j}}$

$idf_{x}=\log{\frac{N}{N_{x}}}$

$TF-IDF_{x,j}=tf_{x,j}*idf_{x}$

x为单词，j为第j个文档，N为文档总数量，$N_{x}$为包含单词x的文档总数量

#### 2. LDA算法的基本假设是什么？

假设语料库中包含K个主题，每篇文档有一个文档-主题分布和主题-词分布。

#### 3. 在TextRank算法中构建图的权重是如何得到的？

如果以词为节点，权重为边两端词的共现次数；如果以句子为节点，权重为边两端句子的相似度。

#### 4. 什么是命名实体识别？ 有什么应用场景？

别文本中具有特定意义的实体，主要包括人名、地名、机构名、专有名词等，是关系抽取、事件抽取、知识图谱、信息提取、问答系统、句法分析、机器翻译等诸多NLP任务的基础。

#### 5.NLP主要有哪几类任务 ？

分类任务，如文本分类、情感分类；生成任务，如机器翻译、问答系统。

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

####  抽取关键字

In [1]:
import jieba
import numpy as np

In [2]:
# 1.导入数据
with open('sample.txt') as f1:
    data = f1.read()
data = data.replace('\n', '')
data = data.replace(' ', '')

In [3]:
with open('stopwords.txt') as f2:
    stopwords = f2.read().split('\n')
# stopwords

In [4]:
import jieba.posseg as psg

def textrank2(document, niters = 10, span=5, topK=20, d=0.85, allowPOS=('ns', 'n', 'vn', 'v')):
    after_cut = [x.word for x in jieba.posseg.cut(document) if x.flag in allowPOS and len(x.word) > 1 and x.word not in stopwords]
    words = list(set(after_cut))
    co_matrix = np.zeros((len(words), len(words))) + 1.0 / len(words)
    ws = np.ones((len(words)))
    
#     共现矩阵
    for i in range(len(after_cut)):
        m = words.index(after_cut[i])
            
        for j in range(i + 1, i + span):
            if j >= len(words):
                break                
            n = words.index(after_cut[j])
            co_matrix[m, n] += 1
                
    for k in range(niters):
#         if k == 0:
        ws = (1 - d) + d * np.dot(ws, 1. / np.sum(co_matrix, axis = 1).reshape(-1, 1) * co_matrix)

    min_size = min(ws)
    max_size = max(ws)
#     print(ws)
    
    results = [(words[i], (ws[i] - min_size) / (max_size - min_size)) for i in range(len(ws))]
    results = sorted(results, key = lambda x : x[1], reverse = True)
    
    return results[:topK]

In [5]:
textrank2(data)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/ng/1yxn2hp94vb65hy2rqh73__c0000gp/T/jieba.cache
Loading model cost 0.927 seconds.
Prefix dict has been built successfully.


[('病例', 1.0),
 ('疫情', 0.9973072422229192),
 ('复工', 0.9895934985149306),
 ('复学', 0.9226446017685198),
 ('感染者', 0.9151776553874014),
 ('风险', 0.8648847467766471),
 ('上海', 0.6335717605689725),
 ('聚集', 0.6325296461897005),
 ('没有', 0.6073076710962794),
 ('武汉', 0.5644594131707368),
 ('口罩', 0.48862754559207183),
 ('可能', 0.4869703046649232),
 ('中国', 0.482833591088373),
 ('控制', 0.4722476958769905),
 ('复产', 0.4688120729329363),
 ('核酸', 0.40147107693877826),
 ('需要', 0.39473945136597927),
 ('保持', 0.3878763785300145),
 ('传播', 0.3690985730014459),
 ('距离', 0.36707770892197167)]

In [6]:
from jieba.analyse import *
textrank(data, withWeight = True)

[('疫情', 1.0),
 ('病例', 0.6057541971353849),
 ('没有', 0.6041099614918801),
 ('感染者', 0.5321139769217231),
 ('复工', 0.5146290559377263),
 ('风险', 0.5051330146452009),
 ('疫苗', 0.4887178555535026),
 ('新冠', 0.47717365987422805),
 ('聚集', 0.4672679646935086),
 ('复学', 0.4666314228793633),
 ('可能', 0.38000092540078684),
 ('群体', 0.3614637932326081),
 ('防控', 0.35666470662798394),
 ('需要', 0.3423462224818712),
 ('免疫', 0.33351942219242287),
 ('病毒', 0.3264158957330911),
 ('武汉', 0.2858046972767431),
 ('控制', 0.2804712528125678),
 ('吃饭', 0.2739091641044968),
 ('中国', 0.26842355750653746)]

In [7]:
# nx.pagerank??

#### 提取关键句

In [7]:
import re
import networkx as nx
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
def cut(document):
    sents = []
    marks = '[。！；？]'
    sentences = re.split(marks, document)[:-1]
    
    for sentence in sentences:
        sentence = ''.join(re.findall(r'[\u4e00-\u9fa5]+', sentence))
        sents.append([x for x in jieba.cut(sentence) if x not in stopwords])
    
    return sents, sentences

In [9]:
sentences_seg, sentences = cut(data)
wordmodel = Word2Vec(sentences_seg, size = 100, min_count=1)

In [31]:
# sentences_seg

In [25]:
# wordmodel2 = Word2Vec.load('../NLP_Project1/CC/model2.model')

In [26]:
# wordmodel.wv["疫情"]

In [10]:
def textrank3(sents, sentences):
    sentences_vector=[]
    n = len(sents)
    
    # 生成平均句向量
    for sen in sents:
        sum_vec = np.zeros((100,))
        for word in sen:
            if word in wordmodel:
                sum_vec += wordmodel.wv[word]
            else:
                print(word, "not in wordmodel.")

        sentences_vector.append(sum_vec/(len(sen) + 1e-5))
    
#     print(sentences_vector)
    
    # 生成句向量的相似度矩阵
    sim_matrix = np.ones((n, n)) + 1.0 / n
    for i in range(n):
        for j in range(n):
            if i != j:
                sim_matrix[i, j] = cosine_similarity(sentences_vector[i].reshape(1,-1), sentences_vector[j].reshape(1,-1))[0,0]

    
    # 手写
    scores = np.ones(n)
    for k in range(100):
        scores = 0.15 + 0.85 * np.dot(scores, 1. / np.sum(sim_matrix, axis = 1).reshape(-1, 1) * sim_matrix)
    
    # 调用nx.pagerank()并排序
#     nx_graph = nx.from_numpy_array(sim_matrix)
# #    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), key = lambda x:x[0], reverse=True)
    
    return ranked_sentences

def print_abstract(ranked_sentences, topn = 5):
    abstract = ''
    num_sentences = min(topn, len(ranked_sentences))
    for i in range(num_sentences):
        abstract += ranked_sentences[i][1]
        abstract += '。'
    print(abstract)
    
    return abstract

In [11]:
results = textrank3(sentences_seg, sentences)
print_abstract(results)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':


张文宏回应外媒指责：如果“隐藏病例数”，武汉敢打开么。如果藏着掖着很多病例，武汉敢打开吗。中国病例数相对较少是因为有隐藏病例。国外有很多媒体说中国的病例数这么少，上海这么少，有没有把病人给藏着掖着。不过，天气热了，病例数是会下降的。


'张文宏回应外媒指责：如果“隐藏病例数”，武汉敢打开么。如果藏着掖着很多病例，武汉敢打开吗。中国病例数相对较少是因为有隐藏病例。国外有很多媒体说中国的病例数这么少，上海这么少，有没有把病人给藏着掖着。不过，天气热了，病例数是会下降的。'

抽取句子经过100轮迭代之后仍未收敛。

#### 选做 1.  提取新闻语料里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

#### 选择2. ： 电影评论分类。

在这个作业中你要完成一个电影评论分类任务。

1.数据获取。（采用爬虫技术爬取相关网页上的电影评论数据，例如猫眼电影评论，豆瓣电影评论）

2.把所获得数据分解为训练集，验证集和测试集。

3.选用相应算法构建模型，并测试。

#### 选择3：文章自动续写

在这个作业中你要完成一个文章自动续写的模型。

1.数据获取。（根据你的兴趣采用爬虫技术爬去相关网站上的文本数据内容：比如故事网站，小说网站等）

2.选取模型，并训练。

3.展示一些你模型的输出例子。